<a href="https://colab.research.google.com/github/Pulkit-g03/Youtube-Thumbnail-Sentiment-Analyzer/blob/main/Thumbnail_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
!pip install fastapi[all] nest-asyncio pyngrok uvicorn streamlit -q

In [15]:
%%shell
sudo apt -y update
sudo apt install -y wget curl unzip
wget http://archive.ubuntu.com/ubuntu/pool/main/libu/libu2f-host/libu2f-udev_1.1.4-1_all.deb
dpkg -i libu2f-udev_1.1.4-1_all.deb
wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
dpkg -i google-chrome-stable_current_amd64.deb
CHROME_DRIVER_VERSION=`curl -sS chromedriver.storage.googleapis.com/LATEST_RELEASE`
wget -N https://chromedriver.storage.googleapis.com/$CHROME_DRIVER_VERSION/chromedriver_linux64.zip -P /tmp/
unzip -o /tmp/chromedriver_linux64.zip -d /tmp/
chmod +x /tmp/chromedriver
mv /tmp/chromedriver /usr/local/bin/chromedriver
pip install selenium


Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:6 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
82 packages can be upgraded. Run 'apt list --upgradable' to see them.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
w

In [16]:
import sys
sys.path.insert(0, '/usr/lib/chromium-browser/chromedriver')

import time
from selenium import webdriver
import chromedriver_autoinstaller

In [28]:
!pip install chromedriver-autoinstaller -q
import sys
sys.path.insert(0, '/usr/lib/chromium-browser/chromedriver')
import time
from selenium import webdriver
import chromedriver_autoinstaller
chrome_options = webdriver.ChromeOptions()

chromedriver_autoinstaller.install()

'/usr/local/lib/python3.10/dist-packages/chromedriver_autoinstaller/121/chromedriver'

In [18]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
import urllib.request
import warnings
warnings.filterwarnings('ignore')

In [19]:
from transformers import pipeline

In [20]:
pipe_Image2Text = pipeline(
    "image-to-text", model="Salesforce/blip-image-captioning-large", device = 0)

pipe_GrammarCorrection = pipeline(
    "text2text-generation", model="pszemraj/flan-t5-large-grammar-synthesis", device = 0)

pipe_SentimentAnalysis = pipeline(
    "text-classification", model="lxyuan/distilbert-base-multilingual-cased-sentiments-student", return_all_scores = True, device = 0)

In [21]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [22]:
!ngrok authtoken 2bZRRtHCKPe9h2FBx4I1L0d7WXw_75Th3Gpx4bkdxKv7V77ks

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [23]:
from fastapi import FastAPI
import nest_asyncio
from pyngrok import ngrok
import uvicorn
from pydantic import BaseModel

app = FastAPI()

class QueryRequest(BaseModel):
  query: str

class SentimentResponse(BaseModel):
  sentiment: str


@app.post('/analyze', response_model = SentimentResponse)
async def Thumbnail_Sentiment(request:QueryRequest):

  qry = request.query.replace(' ', '+')
  driver = webdriver.Chrome(options=chrome_options)
  driver.get(f'https://www.youtube.com/results?search_query={qry}')
  wait = WebDriverWait(driver, 10)


  user_data = driver.find_elements("xpath", '//*[@id="video-title"]')
  links = []
  for i in user_data:
      attr = i.get_attribute("href")
      if attr is not None and r"/shorts/" not in attr:
          a = attr.split('v=')[1]
          a = a.split('&')[0]
          links.append(a)

  for i in range(len(links)):
    urllib.request.urlretrieve( f"https://i.ytimg.com/vi/{links[i]}/maxresdefault.jpg", f"{i}.jpg")

  descriptions = []
  for i in range(len(links)):
    p = pipe_Image2Text(f'{i}.jpg')
    descriptions.append(p)

  sentiments = []
  for i in range(len(descriptions)):
    descriptions[i] = pipe_GrammarCorrection(descriptions[i][0].get('generated_text'))
    corrected_text = descriptions[i][0]["generated_text"]
    sentiments.append(pipe_SentimentAnalysis(corrected_text)[0])


  def overall(sent):
    pos = 0
    neu = 0
    neg = 0

    for s in sent:
        pos += s[0]['score'] if s[0]['label'] == 'positive' else 0
        neu += s[1]['score'] if s[1]['label'] == 'neutral' else 0
        neg += s[2]['score'] if s[2]['label'] == 'negative' else 0


    if pos > neu and pos > neg:
        overall_sentiment = 'positive'
    elif neu > pos and neu > neg:
        overall_sentiment = 'neutral'
    elif neg > pos and neg > neu:
        overall_sentiment = 'negative'
    else:
        overall_sentiment = 'neutral'

    return overall_sentiment

  overall_sentiment = overall(sentiments)

  return SentimentResponse(sentiment = overall_sentiment)

In [25]:
ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port = 8000)

Public URL: https://0c02-34-125-13-145.ngrok-free.app


INFO:     Started server process [360]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     27.0.59.243:0 - "GET / HTTP/1.1" 404 Not Found
INFO:     27.0.59.243:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     27.0.59.243:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     27.0.59.243:0 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     27.0.59.243:0 - "POST /analyze HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [360]


In [26]:
%%writefile main.py

import streamlit as st
import requests
from typing import List

def main():
    st.title("Youtube Thumbnail Sentiment Analyser")

    # Input for the YouTube query
    query = st.text_input("Enter YouTube query")

    if st.button("Analyze"):
        if not query:
            st.warning("Please enter a YouTube query.")
        else:
            response = analyze_thumbnail_sentiment(query)
            st.write(response)
def analyze_thumbnail_sentiment(query: str) -> dict:
    api_url = "http://localhost:8000/analyze"
    payload = {"query": query}
    response = requests.post(api_url, json=payload)
    response.raise_for_status()
    return response.json()

if __name__ == "__main__":
  main()

Writing main.py


In [27]:
ngrok_tunnel = ngrok.connect(80)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
!nohup streamlit run main.py --server.port 80 &
uvicorn.run(app, port=8000)

Exception in thread Thread-14 (_monitor_process):
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.10/dist-packages/pyngrok/process.py", line 140, in _monitor_process
    self._log_line(self.proc.stdout.readline())
  File "/usr/lib/python3.10/encodings/ascii.py", line 26, in decode
    return codecs.ascii_decode(input, self.errors)[0]
UnicodeDecodeError: 'ascii' codec can't decode byte 0xc2 in position 184: ordinal not in range(128)


Public URL: https://d05d-34-125-13-145.ngrok-free.app
nohup: appending output to 'nohup.out'


INFO:     Started server process [360]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:60518 - "POST /analyze HTTP/1.1" 500 Internal Server Error


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/uvicorn/protocols/http/httptools_impl.py", line 419, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
  File "/usr/local/lib/python3.10/dist-packages/uvicorn/middleware/proxy_headers.py", line 84, in __call__
    return await self.app(scope, receive, send)
  File "/usr/local/lib/python3.10/dist-packages/fastapi/applications.py", line 1054, in __call__
    await super().__call__(scope, receive, send)
  File "/usr/local/lib/python3.10/dist-packages/starlette/applications.py", line 123, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/usr/local/lib/python3.10/dist-packages/starlette/middleware/errors.py", line 186, in __call__
    raise exc
  File "/usr/local/lib/python3.10/dist-packages/starlette/middleware/errors.py", line 164, in __call__
    await self.app(scope, receive, _send)
  File "/usr/local/lib/python3.10/dis

INFO:     127.0.0.1:46464 - "POST /analyze HTTP/1.1" 200 OK
INFO:     127.0.0.1:46858 - "POST /analyze HTTP/1.1" 200 OK
INFO:     127.0.0.1:52678 - "POST /analyze HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [360]
